In [31]:
import pandas as pd
import random
from time import sleep
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
import numpy as np
import nltk
import os
from sklearn import datasets
import pickle
from nltk.tokenize import word_tokenize
from sklearn.decomposition import NMF, TruncatedSVD
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.neighbors import NearestNeighbors
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, roc_curve, auc, fbeta_score
from sklearn.model_selection import learning_curve, KFold, GridSearchCV, cross_val_score
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.dummy import DummyClassifier
from collections import Counter
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import text
from sklearn.preprocessing import normalize
import re
from unidecode import unidecode

import matplotlib.style as style
%matplotlib inline
plt.style.use('ggplot')

In [32]:
with open('data/cleaned_df.pkl', 'rb') as picklefile:
    df = pickle.load(picklefile)

In [33]:
df.head()

,song_owner,song_title,song_element,verse,artist
0,Childish Gambino,This Is America,[Bridge: Childish Gambino & Young Thug],"We just wanna party, Party just for you, We ju...",Childish Gambino
1,Childish Gambino,This Is America,[Chorus: Childish Gambino],"This is America, Don't catch you slippin' now,...",Childish Gambino
2,Childish Gambino,This Is America,"[Verse 1: Childish Gambino, Blocboy JB, Slim J...","This is America (skrrt, skrrt, woo), Don't cat...",Childish Gambino
3,Childish Gambino,This Is America,[Refrain: Choir & Childish Gambino],"Ooh-ooh-ooh-ooh-ooh, tell somebody, You go tel...",Childish Gambino
4,Childish Gambino,This Is America,"[Chorus: Childish Gambino, Slim Jxmmi & Young ...","This is America (woo, ayy), Don't catch you sl...",Childish Gambino


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6304 entries, 0 to 6303
Data columns (total 5 columns):
song_owner      6304 non-null object
song_title      6304 non-null object
song_element    6304 non-null object
verse           6304 non-null object
artist          6304 non-null object
dtypes: object(5)
memory usage: 246.3+ KB


In [35]:
model_df = df.copy()

model_df = model_df[model_df['artist'] == 'Childish Gambino']

In [36]:
model_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 824 entries, 0 to 852
Data columns (total 5 columns):
song_owner      824 non-null object
song_title      824 non-null object
song_element    824 non-null object
verse           824 non-null object
artist          824 non-null object
dtypes: object(5)
memory usage: 38.6+ KB


In [37]:
model_df['verse_words'] = model_df['verse'].copy()
model_df['verse_words'] = model_df['verse_words'].str.lower()
model_df['verse_words'] = model_df['verse_words'].apply(lambda x: x.replace(',',''))
model_df['verse_words'] = model_df['verse_words'].apply(lambda x: x.split(' '))

In [38]:
model_df.head()

,song_owner,song_title,song_element,verse,artist,verse_words
0,Childish Gambino,This Is America,[Bridge: Childish Gambino & Young Thug],"We just wanna party, Party just for you, We ju...",Childish Gambino,"[we, just, wanna, party, party, just, for, you..."
1,Childish Gambino,This Is America,[Chorus: Childish Gambino],"This is America, Don't catch you slippin' now,...",Childish Gambino,"[this, is, america, don't, catch, you, slippin..."
2,Childish Gambino,This Is America,"[Verse 1: Childish Gambino, Blocboy JB, Slim J...","This is America (skrrt, skrrt, woo), Don't cat...",Childish Gambino,"[this, is, america, (skrrt, skrrt, woo), don't..."
3,Childish Gambino,This Is America,[Refrain: Choir & Childish Gambino],"Ooh-ooh-ooh-ooh-ooh, tell somebody, You go tel...",Childish Gambino,"[ooh-ooh-ooh-ooh-ooh, tell, somebody, you, go,..."
4,Childish Gambino,This Is America,"[Chorus: Childish Gambino, Slim Jxmmi & Young ...","This is America (woo, ayy), Don't catch you sl...",Childish Gambino,"[this, is, america, (woo, ayy), don't, catch, ..."


In [39]:
from nltk.tokenize import word_tokenize
model_df['tokenize_verse'] = model_df['verse'].copy()
model_df['tokenize_verse'] = model_df['tokenize_verse'].str.lower()
model_df['tokenize_verse'] = model_df['tokenize_verse'].apply(lambda x: word_tokenize(x))

In [40]:
model_df.head()

,song_owner,song_title,song_element,verse,artist,verse_words,tokenize_verse
0,Childish Gambino,This Is America,[Bridge: Childish Gambino & Young Thug],"We just wanna party, Party just for you, We ju...",Childish Gambino,"[we, just, wanna, party, party, just, for, you...","[we, just, wan, na, party, ,, party, just, for..."
1,Childish Gambino,This Is America,[Chorus: Childish Gambino],"This is America, Don't catch you slippin' now,...",Childish Gambino,"[this, is, america, don't, catch, you, slippin...","[this, is, america, ,, do, n't, catch, you, sl..."
2,Childish Gambino,This Is America,"[Verse 1: Childish Gambino, Blocboy JB, Slim J...","This is America (skrrt, skrrt, woo), Don't cat...",Childish Gambino,"[this, is, america, (skrrt, skrrt, woo), don't...","[this, is, america, (, skrrt, ,, skrrt, ,, woo..."
3,Childish Gambino,This Is America,[Refrain: Choir & Childish Gambino],"Ooh-ooh-ooh-ooh-ooh, tell somebody, You go tel...",Childish Gambino,"[ooh-ooh-ooh-ooh-ooh, tell, somebody, you, go,...","[ooh-ooh-ooh-ooh-ooh, ,, tell, somebody, ,, yo..."
4,Childish Gambino,This Is America,"[Chorus: Childish Gambino, Slim Jxmmi & Young ...","This is America (woo, ayy), Don't catch you sl...",Childish Gambino,"[this, is, america, (woo, ayy), don't, catch, ...","[this, is, america, (, woo, ,, ayy, ), ,, do, ..."


In [41]:
model_df['verse'] = model_df['verse'].str.lower()
model_df['verse'] = model_df['verse'].apply(lambda x: x.replace("'",''))
model_df['verse'] = model_df['verse'].apply(lambda x: "".join([c if c.isalnum() else " " for c in x ]))
model_df['verse'] = model_df['verse'].apply(lambda x: x.replace('  ',' '))

In [42]:
stopwords = ["yeah", "uh", "huh", "aw", "oh", "la", "ayy",
            "na", "ha", "ay", "skrrt",
            "em", "bla", "woo", "wu", "ya", "wala", "woop", "wrarrr", "ooh",
            "ba", "duh", "du", "1", "2", "3", "4" ,"5" ,"6", "7", "8", "9", "0"]

for index, row in model_df.iterrows():
    querywords = row['verse'].split()

    resultwords  = [word for word in querywords if word.lower() not in stopwords]
    row['verse'] = ' '.join(resultwords)

model_df.verse[:5]

0    we just wanna party party just for you we just...
1    this is america dont catch you slippin now don...
2    this is america dont catch you slippin now loo...
3    tell somebody you go tell somebody grandma tol...
4    this is america dont catch you slippin now don...
Name: verse, dtype: object

In [43]:
model_df.head()

,song_owner,song_title,song_element,verse,artist,verse_words,tokenize_verse
0,Childish Gambino,This Is America,[Bridge: Childish Gambino & Young Thug],we just wanna party party just for you we just...,Childish Gambino,"[we, just, wanna, party, party, just, for, you...","[we, just, wan, na, party, ,, party, just, for..."
1,Childish Gambino,This Is America,[Chorus: Childish Gambino],this is america dont catch you slippin now don...,Childish Gambino,"[this, is, america, don't, catch, you, slippin...","[this, is, america, ,, do, n't, catch, you, sl..."
2,Childish Gambino,This Is America,"[Verse 1: Childish Gambino, Blocboy JB, Slim J...",this is america dont catch you slippin now loo...,Childish Gambino,"[this, is, america, (skrrt, skrrt, woo), don't...","[this, is, america, (, skrrt, ,, skrrt, ,, woo..."
3,Childish Gambino,This Is America,[Refrain: Choir & Childish Gambino],tell somebody you go tell somebody grandma tol...,Childish Gambino,"[ooh-ooh-ooh-ooh-ooh, tell, somebody, you, go,...","[ooh-ooh-ooh-ooh-ooh, ,, tell, somebody, ,, yo..."
4,Childish Gambino,This Is America,"[Chorus: Childish Gambino, Slim Jxmmi & Young ...",this is america dont catch you slippin now don...,Childish Gambino,"[this, is, america, (woo, ayy), don't, catch, ...","[this, is, america, (, woo, ,, ayy, ), ,, do, ..."


In [44]:
def get_words(df):
    all_words = []
    
    for index, row in df['verse'].iteritems():
        new_words = re.findall(r"\b[a-z']+\b", unidecode(row))
        all_words += new_words
        
    return all_words

In [45]:
model_df.head()

,song_owner,song_title,song_element,verse,artist,verse_words,tokenize_verse
0,Childish Gambino,This Is America,[Bridge: Childish Gambino & Young Thug],we just wanna party party just for you we just...,Childish Gambino,"[we, just, wanna, party, party, just, for, you...","[we, just, wan, na, party, ,, party, just, for..."
1,Childish Gambino,This Is America,[Chorus: Childish Gambino],this is america dont catch you slippin now don...,Childish Gambino,"[this, is, america, don't, catch, you, slippin...","[this, is, america, ,, do, n't, catch, you, sl..."
2,Childish Gambino,This Is America,"[Verse 1: Childish Gambino, Blocboy JB, Slim J...",this is america dont catch you slippin now loo...,Childish Gambino,"[this, is, america, (skrrt, skrrt, woo), don't...","[this, is, america, (, skrrt, ,, skrrt, ,, woo..."
3,Childish Gambino,This Is America,[Refrain: Choir & Childish Gambino],tell somebody you go tell somebody grandma tol...,Childish Gambino,"[ooh-ooh-ooh-ooh-ooh, tell, somebody, you, go,...","[ooh-ooh-ooh-ooh-ooh, ,, tell, somebody, ,, yo..."
4,Childish Gambino,This Is America,"[Chorus: Childish Gambino, Slim Jxmmi & Young ...",this is america dont catch you slippin now don...,Childish Gambino,"[this, is, america, (woo, ayy), don't, catch, ...","[this, is, america, (, woo, ,, ayy, ), ,, do, ..."


In [46]:
get_words(model_df)

['we',
 'just',
 'wanna',
 'party',
 'party',
 'just',
 'for',
 'you',
 'we',
 'just',
 'want',
 'the',
 'money',
 'money',
 'just',
 'for',
 'you',
 'i',
 'know',
 'you',
 'wanna',
 'party',
 'party',
 'just',
 'for',
 'free',
 'girl',
 'you',
 'got',
 'me',
 'dancin',
 'girl',
 'you',
 'got',
 'me',
 'dancin',
 'dance',
 'and',
 'shake',
 'the',
 'frame',
 'we',
 'just',
 'wanna',
 'party',
 'party',
 'just',
 'for',
 'you',
 'we',
 'just',
 'want',
 'the',
 'money',
 'money',
 'just',
 'for',
 'you',
 'you',
 'i',
 'know',
 'you',
 'wanna',
 'party',
 'party',
 'just',
 'for',
 'free',
 'girl',
 'you',
 'got',
 'me',
 'dancin',
 'girl',
 'you',
 'got',
 'me',
 'dancin',
 'dance',
 'and',
 'shake',
 'the',
 'frame',
 'this',
 'is',
 'america',
 'dont',
 'catch',
 'you',
 'slippin',
 'now',
 'dont',
 'catch',
 'you',
 'slippin',
 'now',
 'look',
 'what',
 'im',
 'whippin',
 'now',
 'this',
 'is',
 'america',
 'dont',
 'catch',
 'you',
 'slippin',
 'now',
 'dont',
 'catch',
 'you',
 's

In [47]:
verses = list(model_df.verse.values)

In [48]:
verses[:10]

['we just wanna party party just for you we just want the money money just for you i know you wanna party party just for free girl you got me dancin girl you got me dancin dance and shake the frame we just wanna party party just for you we just want the money money just for you you i know you wanna party party just for free girl you got me dancin girl you got me dancin dance and shake the frame',
 'this is america dont catch you slippin now dont catch you slippin now look what im whippin now this is america dont catch you slippin now dont catch you slippin now look what im whippin now',
 'this is america dont catch you slippin now look how im livin now police be trippin now this is america guns in my area word my area i got the strap i gotta carry ima go into this ugh this is guerilla ima go get the bag or ima get the pad im so cold like im so dope like we gon blow like straight up',
 'tell somebody you go tell somebody grandma told me get your money black man get your black man get yo

In [49]:
def make_word_dictionary(verses):
    following = {}
    count = {}
    for verse in verses:
        words = verse.split()
        for index, word in enumerate(words[:-1]):
            # ensure the word is in the dictionary
            if word not in following:
                following[word] = []
                count[word] = 0

            # append the next word
            following[word].append(words[index + 1])
            count[word] += 1

        if words:
            last_word = words[-1]
            if last_word not in following:
                following[last_word] = []
                count[last_word] = 0
                
    return count, following

In [50]:
count, following = make_word_dictionary(verses)

In [51]:
def create_bigrams(verses):
    bigrams = {}
    for words in verses:
        words = words.split()
        words = [x for x in words if x]
        for w1, w2, w3 in zip(words[:-2], words[1:-1], words[2:]):
            bigram = (w1,w2)
            if bigram not in bigrams:
                bigrams[bigram] = [w3]
            else:
                bigrams[bigram].append(w3)   
    return bigrams

In [52]:
verses_bigrams = create_bigrams(verses)

In [53]:
def generate_text_from_bigram(following, bigram, num_words, seeds):    
    all_bigrams = list(bigram.keys())
    current_bigram = random.choice(seeds)
    text = [current_bigram]
    for _ in range(num_words - 1):
        if current_bigram in all_bigrams:
            
            next_word = random.choice(bigram[current_bigram])
        else:
            if following[text[-1]]:
                next_word = random.choice(following[text[-1]])
            else:
                continue
        text.append(next_word)
        current_bigram = (current_bigram[1], next_word)
    return ' '.join(text)

In [54]:
seeds = ["bitch", "love", "drugs", "cash", "guns", "club", "friends", "paper", "liquor", "apple",
         "blue", "red", "streets", "people", "brother", "cap"]

generated_raps = []
for _ in range(0,100):
    wordlist = generate_text_from_bigram(following, verses_bigrams, 33, seeds)
    generated_raps.append(wordlist)
    print(wordlist)
    print('\n')

red alert its a red alert its a fantasy as long as you are fine anita but i only sport clockers got the uber from her mouth whats my name was disney naw


drugs so go pimp a like on vine makin movies with her friends put it on ebay just be a part of it see the vision youre fucking with but if you wanna


paper i gotta flow like turds and my dude freakin out over a worse fate she on my pole like totem life is a butterfly in your hand in little tokyo and tell


streets like my swagger polaroid you can chill in my heart then i got a house in the bathroom we see him in the house and leave your boyfriend is he wearing somebody


liquor come around when you heard bitch ill fuck her for only a photo now sitting on the risotto girl said that she quiet when my dad called saying son are you eating


brother have the time drop a couple times i used to count me out cause i shoulda known i shoulda known shoulda known shoulda known i shoulda known one love you all my


friends are delirious are you 

cash flow like hawk jones the little bitty one hit me on my grind got to help me you aint got a lust for life my swag is always nice the boy blew


red you lyin like a free throw fuck yall we a team i know what i did it doesnt really matter we ve got all our friends and we re moving in together


friends and its hella dope tell ho i m not bob barker yes i work at abc look at my grandmas house man like cannonball i ain t liking what we gonna do


drugs so i spit real shit dont start no shit dont start no shit there wont be no shit there wont be no shit there wont be no no just say and do


people tell them other kids to feel a spark i don t wanna be that guy but i still hit it like a cylon trying to pass rappers start to copy me dumb


blue too good to be a grown up nigga get popped before his 25th oohh yall niggas need to find i really like what is he wearing somebody jack that fools steeze if


red alert never fake i was working all night if you aint gotta fake shit more green than a 

In [55]:
generated_raps

['red alert its a red alert its a fantasy as long as you are fine anita but i only sport clockers got the uber from her mouth whats my name was disney naw',
 'drugs so go pimp a like on vine makin movies with her friends put it on ebay just be a part of it see the vision youre fucking with but if you wanna',
 'paper i gotta flow like turds and my dude freakin out over a worse fate she on my pole like totem life is a butterfly in your hand in little tokyo and tell',
 'streets like my swagger polaroid you can chill in my heart then i got a house in the bathroom we see him in the house and leave your boyfriend is he wearing somebody',
 'liquor come around when you heard bitch ill fuck her for only a photo now sitting on the risotto girl said that she quiet when my dad called saying son are you eating',
 'brother have the time drop a couple times i used to count me out cause i shoulda known i shoulda known shoulda known shoulda known i shoulda known one love you all my',
 'friends are deli

In [56]:
columns = ['markov_generated_verses']

childish_markov_df = pd.DataFrame(generated_raps, columns=columns)

In [57]:
childish_markov_df['artist'] = 'Childish Gambino'

childish_markov_df

,markov_generated_verses,artist
0,red alert its a red alert its a fantasy as lon...,Childish Gambino
1,drugs so go pimp a like on vine makin movies w...,Childish Gambino
2,paper i gotta flow like turds and my dude frea...,Childish Gambino
3,streets like my swagger polaroid you can chill...,Childish Gambino
4,liquor come around when you heard bitch ill fu...,Childish Gambino
5,brother have the time drop a couple times i us...,Childish Gambino
6,friends are delirious are you saying to it its...,Childish Gambino
7,blue blood women when theyre going out these o...,Childish Gambino
8,liquor a shout out to back all that bullshits ...,Childish Gambino
9,love me i would still sweat five gs back when ...,Childish Gambino


In [58]:
with open('data/childish_markov_raps.pkl', 'wb') as picklefile:
    pickle.dump(childish_markov_df, picklefile)